# Analyse des Aggregates Returns

## Importation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

## Fonctions

In [2]:
def get_OLS(x,y):
    x_with_const = sm.add_constant(x)
    # Modélisation de la régression
    model = sm.OLS(y, x_with_const)
    results = model.fit()
    return results

In [3]:
def bootstrap_bias(data,params,nb) :
    betas = []
    stats = []
    rsquared = []
    for _ in range(nb):
        sample = data.sample(frac=1, replace=True)  # Resample with replacement
        res = get_OLS(sample['SSI'],sample.iloc[:,1])
        betas.append(res.params[1])
        stats.append(res.tvalues[1])
        rsquared.append(res.rsquared_adj)
    bias = np.mean(betas) - params[1]
    pvalue = np.mean(np.abs(stats))
    percentiles = [90, 95, 99]
    r2adj = [np.percentile(rsquared, p) for p in percentiles]
    return bias,pvalue,r2adj

## Données

In [4]:
SSI = pd.read_csv(r'..\data\SSI.csv',index_col=0)
SSI.columns=['SSI']

In [5]:
SP500 = pd.read_excel(r"..\data\sp500.xlsx",index_col=0)[['PX_LAST']]
SP500_lag = SP500
SP500=SP500.reset_index(drop=False).resample('M', on='Date').mean().pct_change()

C:\Users\collo\AppData\Local\Temp\ipykernel_21692\4255522607.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  SP500=SP500.reset_index(drop=False).resample('M', on='Date').mean().pct_change()


## Comtemporaine

Full sample

In [6]:
# Données d'exemple
x = SSI['SSI'][:162]
y = SP500['PX_LAST'][6:168]
x.index = y.index
# Modélisation de la régression
results = get_OLS(x, y)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                PX_LAST   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.1429
Date:                Sun, 14 Apr 2024   Prob (F-statistic):              0.706
Time:                        18:10:52   Log-Likelihood:                 308.99
No. Observations:                 162   AIC:                            -614.0
Df Residuals:                     160   BIC:                            -607.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0063      0.003      2.190      0.0

Post 2009

In [7]:
# Données d'exemple
x = SSI['SSI'][:162][30:]
y = SP500['PX_LAST'][6:168][30:]
x.index = y.index
results = get_OLS(x, y)
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                PX_LAST   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     4.089
Date:                Sun, 14 Apr 2024   Prob (F-statistic):             0.0452
Time:                        18:10:52   Log-Likelihood:                 275.21
No. Observations:                 132   AIC:                            -546.4
Df Residuals:                     130   BIC:                            -540.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0119      0.003      4.307      0.0

## Predictive

### Full sample

In [8]:
# Données d'exemple
x = SSI['SSI'][:162]
y = SP500['PX_LAST'][6:169].shift(-1).dropna()
x.index = y.index
# Modélisation de la régression
results = get_OLS(x, y)
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                PX_LAST   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     89.52
Date:                Sun, 14 Apr 2024   Prob (F-statistic):           3.79e-17
Time:                        18:10:52   Log-Likelihood:                 344.66
No. Observations:                 162   AIC:                            -685.3
Df Residuals:                     160   BIC:                            -679.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0036      0.002      1.565      0.1

### post - 2009

In [9]:
# Données d'exemple
x = SSI['SSI'][:162][30:]
y = SP500['PX_LAST'][6:169][30:].shift(-1).dropna()
x.index = y.index
# Modélisation de la régression
results = get_OLS(x, y)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                PX_LAST   R-squared:                       0.249
Model:                            OLS   Adj. R-squared:                  0.244
Method:                 Least Squares   F-statistic:                     43.22
Date:                Sun, 14 Apr 2024   Prob (F-statistic):           1.08e-09
Time:                        18:10:52   Log-Likelihood:                 292.17
No. Observations:                 132   AIC:                            -580.3
Df Residuals:                     130   BIC:                            -574.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0060      0.002      2.481      0.0

In [10]:
data = pd.DataFrame({
    'SSI': x,
    'PX_LAST': y})
bootstrap_bias(data,results.params,1000)

C:\Users\collo\AppData\Local\Temp\ipykernel_21692\3037007711.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas.append(res.params[1])
C:\Users\collo\AppData\Local\Temp\ipykernel_21692\3037007711.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stats.append(res.tvalues[1])
C:\Users\collo\AppData\Local\Temp\ipykernel_21692\3037007711.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas.append(res.params[1])
C:\Users\col

C:\Users\collo\AppData\Local\Temp\ipykernel_21692\3037007711.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas.append(res.params[1])
C:\Users\collo\AppData\Local\Temp\ipykernel_21692\3037007711.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stats.append(res.tvalues[1])
C:\Users\collo\AppData\Local\Temp\ipykernel_21692\3037007711.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas.append(res.params[1])
C:\Users\col

(0.00040766453104681966,
 6.61674706054893,
 [0.37168766131932574, 0.4143118461221074, 0.47653920828100244])